In [47]:
import csv
from services.zoom import generate_jwt, get_meeting_recordings, set_recording_settings, delete_recording
from dotenv import load_dotenv
from services.google import create_folder, upload_file
import requests

# Load the environment variables from the .env file
load_dotenv()

import os

SAVE_FILE_PATH = 'filtered_data.csv'

ALL_RECORDINGS_PATH = 'data.csv'

api_key = os.environ['ZOOM_API_KEY']
api_secret = os.environ['ZOOM_API_SECRET']


# Replace FOLDER_ID with the ID of the folder where you want to upload the file
FOLDER_ID = '19_SkohMvIP9xbw3lzqAQJm5CJNqJq55K'



In [49]:
def read_csv(filename, meeting_key='ID de la reunión') -> list:
    data = list()
    with open(filename, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                """ Tema de la reunión, ID de la reunión, Hora de inicio de la reunión, Tamaño del archivo, Visualizaciones en total, Descargas en total, Último acceso """
                meeting_id = row[meeting_key].replace(' ', '')
                
                data.append(meeting_id)
                line_count += 1
        print(f'Processed {line_count} lines.')
    return data

In [24]:
def write_csv(data, save_file_path):
    with open(save_file_path, mode='w') as save_file:
        filtered_data_writer = csv.writer(save_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        filtered_data_writer.writerow(['Meeting ID'])
        for meeting_id in data:
            filtered_data_writer.writerow([meeting_id])

Obtener Info de los recordings

In [5]:
def get_recordings(meeting_id):
    token = generate_jwt(api_key, api_secret)
    recordings = get_meeting_recordings(token, meeting_id)
    return recordings

def update_recording_settings(meeting_id):
    token = generate_jwt(api_key, api_secret)
    set_recording_settings(token, meeting_id)

In [7]:

def download_file(recording_url, filename):
    token = generate_jwt(api_key, api_secret)
    new_download_url = recording_url +'?access_token=' + token
        
    try:
        response = requests.get(new_download_url, stream=True)
    except requests.exceptions.ConnectionError as err:
        print(err)
        response = requests.get(new_download_url, stream=True)
    try:
        with open(filename, 'wb') as fd:
            for chunk in response.iter_content(chunk_size=1024 * 8):
                if chunk:
                    fd.write(chunk)
                    fd.flush()
                    os.fsync(fd.fileno())
        print(f'File downloaded and saved to {filename}')
        return True    
    except Exception as e:
        print(e)
        return False


In [20]:
def process_meeting_id(meeting_id):
    recording = get_recordings(meeting_id)

    if 'code' in recording:
      if recording['code'] == 3301:
        print("ERROR: Couldn't find recordings for " + meeting_id)
        return False
    
    # Add allow download recording setting for meeting_id
    update_recording_settings(meeting_id)
    
    # Create GDrive folder
    title = recording['start_time'] +'-' + recording['topic']
    folder_id = create_folder(title, FOLDER_ID)

    
    # Process each recording file of specific meeting_id
    recording_files = recording['recording_files']

    for recording in recording_files:
        download_url = recording['download_url']

        file_extension = recording['file_extension']

        if file_extension == 'MP4':
            MIME_TYPE = 'video/mp4'
        elif file_extension == 'TXT':
            MIME_TYPE = 'text/plain'
        elif file_extension == 'M4A':
            MIME_TYPE = 'audio/mpeg'

        filename = recording['id'] + '.' + file_extension

        #download the file
        downloaded = download_file(download_url, filename_path)
        file_size = recording['file_size']/1000/1024
        
        if downloaded:
          print(f'-- Successfully downloaded {filename} size: {file_size} Mb')
        
        if downloaded:
            upload_file(filename, MIME_TYPE, folder_id)

            
            # Delete the file from the local storage
            os.remove(filename)

# Run
Download the csv files of all recording in your Recordings page in zoom and rename it to data.csv

In [ ]:
data = read_csv('data.csv')
total = len(data)

for idx, meeting_id in enumerate(data):
    print(f""" 
    ---------------------------
    {idx+1}/{total} Start meeting: {meeting_id}
    ---------------------------

    """)
    process_meeting_id(meeting_id)
    token = generate_jwt(api_key, api_secret)
    response = delete_recording(meeting_id, token)
    if response == '':
        print(f'{meeting_id} deleted successfully') 
    else:
        print(f""" 
          ---------------------------
          ERROR. probably {meeting_id} couldn't be deleted
          ---------------------------""")